In [10]:
import xarray as xr
from modules import unity_files
import numpy as np
from datetime import timedelta
import modules.data_utils as ut
import os
from pint import UnitRegistry
from tqdm import tqdm

units = UnitRegistry()

In [2]:
data = xr.open_dataset(r"C:\Users\mpvossen\OneDrive - UWM\SCSU\cm1out.nc")
data

<xarray.Dataset>
Dimensions:  (one: 1, time: 9, xh: 300, xf: 301, yh: 300, yf: 301, zh: 55,
              zf: 56)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:00 00:15:00 ... 01:45:00 02:00:00
  * xh       (xh) float32 -74.75 -74.25 -73.75 -73.25 ... 73.75 74.25 74.75
  * xf       (xf) float32 -75.0 -74.5 -74.0 -73.5 -73.0 ... 73.5 74.0 74.5 75.0
  * yh       (yh) float32 -74.75 -74.25 -73.75 -73.25 ... 73.75 74.25 74.75
  * yf       (yf) float32 -75.0 -74.5 -74.0 -73.5 -73.0 ... 73.5 74.0 74.5 75.0
  * zh       (zh) float32 0.2 0.6 1.0 1.4 1.8 2.2 ... 20.2 20.6 21.0 21.4 21.8
  * zf       (zf) float32 0.0 0.4 0.8 1.2 1.6 2.0 ... 20.4 20.8 21.2 21.6 22.0
Dimensions without coordinates: one
Data variables: (12/51)
    ztop     (one) float32 ...
    umove    (time) float32 ...
    vmove    (time) float32 ...
    rain     (time, yh, xh) float32 ...
    prate    (time, yh, xh) float32 ...
    sws      (time, yh, xh) float32 ...
    ...       ...
    w        (time, zf, yh, xh) float32 ...
    tke      (time, zf, yh, xh) float32 ...
    kmh      (time, zf, yh, xh) float32 ...
    kmv      (time, zf, yh, xh) float32 ...
    khh      (time, zf, yh, xh) float32 ...
    khv      (time, zf, yh, xh) float32 ...
Attributes: (12/71)
    CM1 version:    cm1r21.0
    Conventions:    CF-1.7
    missing_value:  -999999.9
    x_units:        km
    x_label:        x
    y_units:        km
    ...             ...
    cgt1:           1.875
    cgt2:           -1.25
    cgt3:           0.375
    dgt1:           2.0
    dgt2:           -3.0
    dgt3:           1.0

In [3]:
data_sel = data.sel(time=timedelta(hours=2))
data_sel

<xarray.Dataset>
Dimensions:  (one: 1, xh: 300, xf: 301, yh: 300, yf: 301, zh: 55, zf: 56)
Coordinates:
    time     timedelta64[ns] 02:00:00
  * xh       (xh) float32 -74.75 -74.25 -73.75 -73.25 ... 73.75 74.25 74.75
  * xf       (xf) float32 -75.0 -74.5 -74.0 -73.5 -73.0 ... 73.5 74.0 74.5 75.0
  * yh       (yh) float32 -74.75 -74.25 -73.75 -73.25 ... 73.75 74.25 74.75
  * yf       (yf) float32 -75.0 -74.5 -74.0 -73.5 -73.0 ... 73.5 74.0 74.5 75.0
  * zh       (zh) float32 0.2 0.6 1.0 1.4 1.8 2.2 ... 20.2 20.6 21.0 21.4 21.8
  * zf       (zf) float32 0.0 0.4 0.8 1.2 1.6 2.0 ... 20.4 20.8 21.2 21.6 22.0
Dimensions without coordinates: one
Data variables: (12/51)
    ztop     (one) float32 ...
    umove    float32 ...
    vmove    float32 ...
    rain     (yh, xh) float32 ...
    prate    (yh, xh) float32 ...
    sws      (yh, xh) float32 ...
    ...       ...
    w        (zf, yh, xh) float32 ...
    tke      (zf, yh, xh) float32 ...
    kmh      (zf, yh, xh) float32 ...
    kmv      (zf, yh, xh) float32 ...
    khh      (zf, yh, xh) float32 ...
    khv      (zf, yh, xh) float32 ...
Attributes: (12/71)
    CM1 version:    cm1r21.0
    Conventions:    CF-1.7
    missing_value:  -999999.9
    x_units:        km
    x_label:        x
    y_units:        km
    ...             ...
    cgt1:           1.875
    cgt2:           -1.25
    cgt3:           0.375
    dgt1:           2.0
    dgt2:           -3.0
    dgt3:           1.0

In [4]:
time = data_sel.time.values
time_s = int(time)/ 10 **9

In [5]:
isosurfaces =  np.arange(20,65,5)
uf = unity_files.unity_files()
y,z,x = np.meshgrid(data_sel.yh.values,data_sel.zh.values, data_sel.xh.values)

print(np.shape(z))
print(np.shape(data_sel.winterp.values))


(55, 300, 300)
(55, 300, 300)


In [6]:
uf.input_isosurface_data(
    x * units('km'),
    y * units('km'),
    z * units('km'),
    data_sel.dbz.values,
    isosurfaces,
    time_s * units.seconds,
    variable_name="simulated reflectivity"    
)

In [7]:
uf.input_vector_data(
    x * units('km'),
    y * units('km'),
    z * units('km'),
    data_sel.uinterp.values * units.mps,
    data_sel.vinterp.values * units.mps,
    data_sel.winterp.values * units.mps,
    time = time_s * units.seconds
)

In [8]:
save_directory = r"C:\Users\mpvossen\OneDrive - UWM\SCSU\Isosurfaces\spilt_ss_new" + os.sep
dir = save_directory + str(int(time_s)) + os.sep
os.makedirs(dir, exist_ok=True)
uf.create_files(dir)

In [ ]:
unity_files.save_vector_field(data_sel.uinterp.values,data_sel.vinterp.values, data_sel.winterp.values, r"C:\Users\mpvos\OneDrive - UWM\SCSU\isosurfaces\split.vf")

In [60]:
np.float128

AttributeError: module 'numpy' has no attribute 'float128'

In [11]:
save_directory = r"C:\Users\mpvossen\OneDrive - UWM\SCSU\Isosurfaces\spilt_ss_new" + os.sep
for time in tqdm(data.time.values[1:]):
    
    data_sel = data.sel(time=time)
    time = data_sel.time.values
    time_s = int(time)/ 10 **9
    isosurfaces =  np.arange(20,65,5)
    uf = unity_files.unity_files()
    y,z,x = np.meshgrid(data_sel.yh.values,data_sel.zh.values, data_sel.xh.values)
    
    uf.input_isosurface_data(
        x * units('km'),
        y * units('km'),
        z * units('km'),
        data_sel.dbz.values,
        isosurfaces,
        time_s * units.seconds,
        variable_name="simulated reflectivity"    
    )
        
    uf.input_vector_data(
        x * units('km'),
        y * units('km'),
        z * units('km'),
        data_sel.uinterp.values * units.mps,
        data_sel.vinterp.values * units.mps,
        data_sel.winterp.values * units.mps,
        time = time_s * units.seconds
    )
    dir = save_directory + str(int(time_s)) + os.sep
    os.makedirs(dir, exist_ok=True)
    uf.create_files(dir)
    del uf, dir
    

100%|██████████| 8/8 [03:06<00:00, 23.26s/it]
